In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math
import networkx as nx
import random 
import itertools as it
import datetime
import os
import multiprocessing as mp
import pyspark.sql.types as types
import pyspark.sql.functions as sf
import pyspark.sql
import pyspark
from pyspark.sql import SparkSession
import re

In [2]:
def wrapper(kpi,weight,selected_quarter):
    list_baseline = ['_Hired_Language_Site_Baseline.Q2_2020.csv',
                     '_Tenure_Site_Baseline.Q2_2020.csv',
                     '_Scheduled_Language_Site_Baseline.Q2_2020.csv', 
                     '_Shift_Pattern_Site_Baseline.Q2_2020.csv',
                     '_Offered_Language_Site_Baseline.Q2_2020.csv',
                     '_Activity_Channel_Site_Baseline.Q2_2020.csv',
                     '_Site_Actuals_2018_2019.csv']
    
    sites = ['AMS14','AMS22','BCN2','BER3','CBG3','LHR5','LIL2','VNO3']
    
    years = [2018,2019,2020]
    
    kpi_ = kpi.split('/')[0]
    
    df = pd.DataFrame()
    print kpi_
    
    for baseline in list_baseline:
        path = str(kpi)+str(baseline)
        try:
            
            if baseline == '_Site_Actuals_2018_2019.csv':
                
                actual_df = pd.read_csv(path)
                actual_df = actual_df.drop(columns = ['Unnamed: 0'])

                actual_df_2018 = actual_df[actual_df.Year == 2018.]
                actual_df_2019 = actual_df[actual_df.Year == 2019.]

                actual_df_2018_Q1 = actual_df_2018[actual_df_2018.Quarter == selected_quarter]
                actual_df_2019_Q1 = actual_df_2019[actual_df_2019.Quarter == selected_quarter]
                
                
                actual_df_2018_Q1 = actual_df_2018_Q1
                actual_df_2019_Q1 = actual_df_2019_Q1

                actual_df_2018_Q1['KPI'] = [kpi_]
                actual_df_2019_Q1['KPI'] = [kpi_]

                actual_df_2018_Q1['Factor'] = ['Actual Performance']
                actual_df_2019_Q1['Factor'] = ['Actual Performance']

                df = pd.concat([df, actual_df_2018_Q1, actual_df_2019_Q1])
                
               
            else:
            
                temp = pd.read_csv(path)
                temp = temp.drop(columns = ['Unnamed: 0'])

                temp['KPI'] = [kpi_]

                factor = str(baseline.split('_Site')[0])[1:] 
                temp['Factor'] = [factor]

                temp['Year'] = [2020]
                temp['Quarter'] = [selected_quarter]

                df = pd.concat([df, temp])
                       
        except:
            print path, 'NOT PRESENT'
        
        
    # WEIGHT LAST YEAR QUARTER WITH NEXT YEAR EXPECTED PERFORMANCE

    df_2020 = df[df.Year == 2020]
    for site in sites:
        temp = []
        for i in range(df_2020.shape[0]):
            temp.append(weight*df_2020[site].iloc[i] + (1-weight)*actual_df_2019_Q1[site].values[0])    
        df_2020[site] = temp
    
    # ADD MEAN OF ALL FACTORS PER SITE PER QUARTER 
    
    mean_df = pd.DataFrame(df_2020.mean(0)).T
    mean_df['KPI'] = [kpi_]
    mean_df['Factor'] = ['Baseline']         
    mean_df['Year'] = [2020]
    mean_df['Quarter'] = [selected_quarter]
    

    mean_df_target = pd.DataFrame(df_2020.mean(0)).T
    mean_df_target['KPI'] = [kpi_]
    mean_df_target['Factor'] = ['Target']         
    mean_df_target['Year'] = [2020]
    mean_df_target['Quarter'] = [selected_quarter]
    
    
    print mean_df_target

    # HARDCODED TARGET ---> ALESSANDRO REQUEST ----> It is going to happen even in the future
    
    '''
    if (kpi_ == 'ADHERENCE'):
        
        mean_df_target = pd.DataFrame(df_2020.mean(0)).T
        mean_df_target['KPI'] = [kpi_]
        mean_df_target['Factor'] = ['Target']         
        mean_df_target['Year'] = [2020]
        mean_df_target['Quarter'] = [selected_quarter]
        
        print mean_df_target
    
    elif (kpi_ == 'CPI-LH'):
        
        mean_df_target = pd.DataFrame()
        for site in sites:
            mean_df_target[site] = [4.]
        mean_df_target['KPI'] = [kpi_]
        mean_df_target['Factor'] = ['Target']         
        mean_df_target['Year'] = [2020]
        mean_df_target['Quarter'] = [selected_quarter]
        
        print mean_df_target
    
    elif (kpi_ == 'CPI-PT'):
        
        mean_df_target = pd.DataFrame(df_2020.mean(0)).T
        mean_df_target['KPI'] = [kpi_]
        mean_df_target['Factor'] = ['Target']         
        mean_df_target['Year'] = [2020]
        mean_df_target['Quarter'] = [selected_quarter]
        
        print mean_df_target
    
    elif (kpi_ == 'FT'):  
    
        mean_df_target = pd.DataFrame(df_2020.mean(0)).T
        mean_df_target['BER3'] = [0.11]
        mean_df_target['KPI'] = [kpi_]
        mean_df_target['Factor'] = ['Target']         
        mean_df_target['Year'] = [2020]
        mean_df_target['Quarter'] = [selected_quarter]
        
        print mean_df_target
    
    elif (kpi_ == 'PSAT'):
        
        mean_df_target = pd.DataFrame()
        for site in sites:
            if site!='VNO3':
                mean_df_target[site] = [0.9]
            else:
                mean_df_target[site] = [0.885]
                
        mean_df_target['KPI'] = [kpi_]
        mean_df_target['Factor'] = ['Target']         
        mean_df_target['Year'] = [2020]
        mean_df_target['Quarter'] = [selected_quarter]
        
        print mean_df_target
        
    elif (kpi_ == 'QS'):
        
        dic = {'AMS14':[0.93],'AMS22':[0.93],'BCN2':[0.93],'BER3':[0.92],'CBG3':[0.92],'LHR5':[0.93],'LIL2':[0.92],
               'VNO3':[0.93]}
        mean_df_target = pd.DataFrame.from_dict(dic)
        mean_df_target['KPI'] = [kpi_]
        mean_df_target['Factor'] = ['Target']         
        mean_df_target['Year'] = [2020]
        mean_df_target['Quarter'] = [selected_quarter]
        
        print mean_df_target
    '''
    
    df = pd.concat([df, mean_df, mean_df_target])

    # ADD MEAN OF ALL FACTORS OF ALL SITES PER QUARTER 
    q1_abs_summary_df, q1_rel_diff_summary_df = pd.DataFrame(), pd.DataFrame()
    
    temp_2018, temp_2019, temp_2020 = [],[],[]
        
    for site in sites:
        temp_2018.append(df[df.Year == 2018][site].values[0])
        temp_2019.append(df[df.Year == 2019][site].values[0])
        temp_2020.append(mean_df_target[site].values[0])

    
    temp_2018_df  = pd.DataFrame([np.mean(np.mean([i for i in temp_2018 if str(i) != 'nan']))])
    temp_2018_df['KPI'] = [kpi_]       
    temp_2018_df['Year'] = [2018]
    temp_2018_df['Quarter'] = [selected_quarter]

    
    temp_2019_df  = pd.DataFrame([np.mean(np.mean([i for i in temp_2019 if str(i) != 'nan']))]).T
    temp_2019_df['KPI'] = [kpi_]       
    temp_2019_df['Year'] = [2019]
    temp_2019_df['Quarter'] = [selected_quarter]
    
    temp_2020_df  = pd.DataFrame([np.mean(np.mean([i for i in temp_2020 if str(i) != 'nan']))]).T
    temp_2020_df['KPI'] = [kpi_]       
    temp_2020_df['Year'] = [2020]
    temp_2020_df['Quarter'] = [selected_quarter]

    q1_abs_summary_df = pd.concat([temp_2018_df, temp_2019_df, temp_2020_df])
    
    
    abs_2020 = q1_abs_summary_df[q1_abs_summary_df.Year == 2020][0]
    abs_2019 = q1_abs_summary_df[q1_abs_summary_df.Year == 2019][0]
    abs_2018 = q1_abs_summary_df[q1_abs_summary_df.Year == 2018][0]
    
    # RELATIVE DIFFERENCES BETWEEN YEARS
    
    if (kpi_ == 'FT'):
        rel_diff_2018 = -(abs_2020 - abs_2018)
        
    elif (kpi_ == 'CPI-LH' or kpi_ == 'CPI-PT'):
        
        rel_diff_2018 = +(abs_2020 - abs_2018)/abs_2018
        
    else:
        rel_diff_2018 = +(abs_2020 - abs_2018)
        
    temp_df_2018  = pd.DataFrame([rel_diff_2018]).T
    temp_df_2018['KPI'] = [kpi_]       
    temp_df_2018['Year'] = ['2020_2018']
    
    if (kpi_ == 'FT'):
        rel_diff_2019 = -(abs_2020 - abs_2019)
        
    elif (kpi_ == 'CPI-LH' or kpi_ == 'CPI-PT'):
        rel_diff_2019 = +(abs_2020 - abs_2019)/abs_2019
        
    else:
        rel_diff_2019 = +(abs_2020 - abs_2019)
    

    temp_df_2019  = pd.DataFrame([rel_diff_2019]).T
    temp_df_2019['KPI'] = [kpi_]       
    temp_df_2019['Year'] = ['2020_2019']
    
    q1_rel_diff_summary_df = pd.concat([ temp_df_2019,  temp_df_2018])
    
    return df, q1_abs_summary_df, q1_rel_diff_summary_df

KPIs_folder = ['ADHERENCE','CPI-LH','CPI-PT','FT','PSAT','QS']

final_df, final_abs_summary_df, final_rel_diff_summary_df = pd.DataFrame(),pd.DataFrame(),pd.DataFrame()

weight = 0.5
selected_quarter = 2
year = 2020
for kpi in KPIs_folder:
    path = str(kpi)+'/'+str(kpi)
    df, abs_summary_df, rel_diff_summary_df = wrapper(path,weight,selected_quarter)
    
    final_df  = pd.concat([final_df, df])
    final_abs_summary_df  = pd.concat([final_abs_summary_df, abs_summary_df])
    final_rel_diff_summary_df  = pd.concat([final_rel_diff_summary_df, rel_diff_summary_df])

    
final_abs_summary_df.to_csv('Q2_2020_abs_summary.csv')
final_rel_diff_summary_df.to_csv('Q2_2020_rel_diff_summary.csv')

ADHERENCE
ADHERENCE/ADHERENCE_Offered_Language_Site_Baseline.Q2_2020.csv NOT PRESENT


/opt/blue-python/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/blue-python/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/blue-python/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/in

      AMS14     AMS22      BCN2      BER3      CBG3      LHR5      LIL2  \
0  0.924191  0.922275  0.921645  0.911464  0.926588  0.923097  0.919292   

   Quarter      VNO3  Year        KPI  Factor  
0        2  0.929089  2020  ADHERENCE  Target  
CPI-LH


/opt/blue-python/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:168: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



CPI-LH/CPI-LH_Offered_Language_Site_Baseline.Q2_2020.csv NOT PRESENT


/opt/blue-python/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:60: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



      AMS14     AMS22      BCN2      BER3      CBG3      LHR5      LIL2  \
0  3.535032  3.526026  3.684855  3.555796  3.549071  3.667641  3.470473   

   Quarter      VNO3  Year     KPI  Factor  
0        2  3.364921  2020  CPI-LH  Target  
CPI-PT
CPI-PT/CPI-PT_Hired_Language_Site_Baseline.Q2_2020.csv NOT PRESENT
CPI-PT/CPI-PT_Tenure_Site_Baseline.Q2_2020.csv NOT PRESENT
CPI-PT/CPI-PT_Scheduled_Language_Site_Baseline.Q2_2020.csv NOT PRESENT
CPI-PT/CPI-PT_Shift_Pattern_Site_Baseline.Q2_2020.csv NOT PRESENT
CPI-PT/CPI-PT_Offered_Language_Site_Baseline.Q2_2020.csv NOT PRESENT
CPI-PT/CPI-PT_Activity_Channel_Site_Baseline.Q2_2020.csv NOT PRESENT
   Year  Quarter  AMS14  AMS22  BCN2  BER3  CBG3  LHR5  LIL2  VNO3     KPI  \
0  2020        2    NaN    NaN   NaN   NaN   NaN   NaN   NaN   NaN  CPI-PT   

   Factor  
0  Target  
FT
FT/FT_Offered_Language_Site_Baseline.Q2_2020.csv NOT PRESENT


/opt/blue-python/2.7/lib/python2.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/opt/blue-python/2.7/lib/python2.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/opt/blue-python/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:248: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



      AMS14     AMS22      BCN2      BER3      CBG3      LHR5      LIL2  \
0  0.105192  0.098468  0.099183  0.114703  0.108755  0.099654  0.096468   

   Quarter      VNO3  Year KPI  Factor  
0        2  0.107972  2020  FT  Target  
PSAT
PSAT/PSAT_Scheduled_Language_Site_Baseline.Q2_2020.csv NOT PRESENT
     AMS14     AMS22      BCN2      BER3      CBG3      LHR5      LIL2  \
0  0.88708  0.879932  0.887995  0.892732  0.889849  0.890449  0.886114   

   Quarter      VNO3  Year   KPI  Factor  
0        2  0.872977  2020  PSAT  Target  
QS
QS/QS_Offered_Language_Site_Baseline.Q2_2020.csv NOT PRESENT
      AMS14     AMS22      BCN2      BER3      CBG3     LHR5      LIL2  \
0  0.930948  0.934225  0.935036  0.921047  0.920651  0.92672  0.917506   

   Quarter      VNO3  Year KPI  Factor  
0        2  0.928103  2020  QS  Target  


In [3]:
#  DIFFERENT STRATEGY FOR CSAT, CSAT_PHONE AND CSAT_NON_PHONE

#for kpi_ in ['CSAT','CSAT_PHONE', 'CSAT_NON_PHONE']:
 
selected_quarter = 2
year = 2020

for kpi_ in ['CSAT']:    
    
    path  = str(kpi_)+'/'+str(kpi_)+str('_Site_Actuals.Q1_2019_2020.csv')
    path1 = str(kpi_)+'/'+str(kpi_)+str('_Site_Actuals.Q1_2019_2020_seasonality.csv')
    path2 = str(kpi_)+'/'+str(kpi_)+str('_Offered_Language_Site_Baseline.Q1_2020.csv')
    path3 = str(kpi_)+'/'+str(kpi_)+str('_Hired_Language_Site_Baseline.Q1_2020.csv')
    path4 = str(kpi_)+'/'+str(kpi_)+str('_Tenure_Site_Baseline.Q1_2020.csv')
    path5 = str(kpi_)+'/'+str(kpi_)+str('_Activity_Channel_Site_Baseline.Q1_2020.csv')
    

    actual_df          = pd.read_csv(path)
    actual_df_seas     = pd.read_csv(path1)
    actual_df_language = pd.read_csv(path2)
    actual_df_h_lang   = pd.read_csv(path3)
    actual_df_tenure   = pd.read_csv(path4)
    actual_df_activity = pd.read_csv(path5)
    
    actual_df          = actual_df.drop(columns = ['Unnamed: 0'])
    actual_df_seas     = actual_df_seas.drop(columns = ['Unnamed: 0'])
    actual_df_language = actual_df_language.drop(columns = ['Unnamed: 0'])
    actual_df_h_lang   = actual_df_h_lang.drop(columns = ['Unnamed: 0'])
    actual_df_tenure   = actual_df_tenure.drop(columns = ['Unnamed: 0'])
    actual_df_activity = actual_df_activity.drop(columns = ['Unnamed: 0'])
    
    print actual_df_language 
    
    actual_df_baseline = 0.5*actual_df.iloc[1,:] + 0.5*actual_df_language
    
    #actual_df_new = 0.5*actual_df.iloc[1,:] + 0.5*actual_df_language + actual_df_seas
    
    actual_df_new = (actual_df.iloc[1,:]+actual_df_language+actual_df_h_lang+actual_df_tenure+actual_df_activity)/5.
    actual_df_new_5pp = actual_df_new*1.05
    actual_df_new_increment_topic = actual_df_new*1.0075
    
    print actual_df_new_5pp

    actual_df_2019 = pd.DataFrame(actual_df.iloc[0,:]).T
    actual_df_2019['Quarter'] = [4]
    actual_df_2019['Year']    = [2019] 
    actual_df_2019['KPI']     = [kpi_]
    actual_df_2019['Factor']  = ['Actual (December 2019)']
    
    print  '\n'
    print 'HIST \n'

    print actual_df_2019
    print  '\n'
    
    actual_df_2020 = pd.DataFrame(actual_df.iloc[1,:]).T
    actual_df_2020['Quarter'] = [1]
    actual_df_2020['Year']    = [year] 
    actual_df_2020['KPI']     = [kpi_]
    actual_df_2020['Factor']  = ['Actual (January 2020)']
    
    print  '\n'
    print 'HIST \n'

    print actual_df_2020
    print  '\n'
    
    actual_df_language['Quarter'] = [1]
    actual_df_language['Year']    = [year]  
    actual_df_language['KPI']     = [kpi_]
    actual_df_language['Factor']  = ['Offered_Language']
    
    print  '\n'
    print 'LANGUAGE \n'
    
    actual_df_h_lang['Quarter'] = [1]
    actual_df_h_lang['Year']    = [year]  
    actual_df_h_lang['KPI']     = [kpi_]
    actual_df_h_lang['Factor']  = ['Hired_Language']
    
    print  '\n'
    print 'LANGUAGE \n'
    
    actual_df_tenure['Quarter'] = [1]
    actual_df_tenure['Year']    = [year]  
    actual_df_tenure['KPI']     = [kpi_]
    actual_df_tenure['Factor']  = ['Tenure']
    
    print  '\n'
    print 'LANGUAGE \n'

    
    actual_df_activity['Quarter'] = [1]
    actual_df_activity['Year']    = [year]  
    actual_df_activity['KPI']     = [kpi_]
    actual_df_activity['Factor']  = ['Activity_Channel']
    
    print  '\n'
    print 'LANGUAGE \n'

    
    print actual_df_language
    print  '\n'

    actual_df_seas['Quarter'] = [1]
    actual_df_seas['Year']    = [year] 
    actual_df_seas['KPI']     = [kpi_]
    actual_df_seas['Factor']  = ['CSAT 2.0 Seasonality (Jan 2020 - Dec 2019)']
    
    print 'SES \n'
    print actual_df_seas
    print  '\n'
    
    actual_df_baseline['Quarter'] = pd.Series([selected_quarter])
    actual_df_baseline['Year']    = pd.Series([year]) 
    actual_df_baseline['KPI']     = [kpi_]
    actual_df_baseline['Factor']  = ['Baseline']
    
    print 'BASE \n'
    print actual_df_baseline
    print  '\n'
    
    actual_df_new['Quarter'] = pd.Series([selected_quarter])
    actual_df_new['Year']    = pd.Series([year]) 
    actual_df_new['KPI']     = [kpi_]
    actual_df_new['Factor']  = ['Baseline']
    
    print 'NEW \n'
    print actual_df_new
    print  '\n'
    
    actual_df_new_5pp['Quarter'] = pd.Series([selected_quarter])
    actual_df_new_5pp['Year']    = pd.Series([year]) 
    actual_df_new_5pp['KPI']     = [kpi_]
    actual_df_new_5pp['Factor']  = ['5% increment overall']
    
    print 'NEW \n'
    print actual_df_new_5pp
    print  '\n'
    
    actual_df_new_increment_topic['Quarter'] = pd.Series([selected_quarter])
    actual_df_new_increment_topic['Year']    = pd.Series([year]) 
    actual_df_new_increment_topic['KPI']     = [kpi_]
    actual_df_new_increment_topic['Factor']  = ['5% increment on Relocation and Incorrect Charges']
    
    print 'NEW \n'
    print actual_df_new_increment_topic
    print  '\n'
    '''
    # add a target blank row
    
    actual_df_new_t = actual_df.copy()
    actual_df_new_t.iloc[:,2:-2] = actual_df_new_t.iloc[:,2:-2]*np.nan
    
    print 'NEW \n'
    print actual_df_new_t
    print  '\n'
    
    actual_df_new_t['Quarter_new'] = pd.Series([selected_quarter])
    actual_df_new_t['Year_new']    = pd.Series([2020])
    actual_df_new_t['Quarter']     = actual_df_new_t['Quarter_new']
    actual_df_new_t['Year']        = actual_df_new_t['Year_new'] 
    actual_df_new_t = actual_df_new_t.drop(columns = ['Quarter_new','Year_new'])

    actual_df_new_t['KPI']= [kpi_]
    actual_df_new_t['Factor']= ['Target']
    
    print 'NEW \n'
    print actual_df_new_t
    print  '\n'
    '''
    
    final_df = pd.concat([final_df,
                          actual_df_2019,
                          actual_df_language,
                          actual_df_h_lang,
                          actual_df_tenure,
                          actual_df_activity,
                          actual_df_new,
                          actual_df_new_5pp,
                          actual_df_new_increment_topic]
                         ,axis=0)

      AMS14    AMS22      BCN2      BER3      CBG3     LHR5     LIL2      VNO3
0  0.854287  0.85047  0.847321  0.856512  0.838765  0.83764  0.84016  0.818693
      AMS14     AMS22      BCN2      BER3      CBG3      LHR5      LIL2  \
0  0.892853  0.892339  0.889776  0.888422  0.873716  0.874716  0.885185   

       VNO3  
0  0.869914  


HIST 

      AMS14     AMS22      BCN2      BER3      CBG3      LHR5      LIL2  \
0  0.863118  0.861436  0.854353  0.846867  0.835443  0.805635  0.858978   

       VNO3  Quarter  Year   KPI                  Factor  
0  0.829072        4  2019  CSAT  Actual (December 2019)  




HIST 

      AMS14     AMS22      BCN2      BER3      CBG3      LHR5      LIL2  \
1  0.854378  0.865193  0.845779  0.835424  0.821497  0.799118  0.847829   

      VNO3  Quarter  Year   KPI                 Factor  
1  0.80798        1  2020  CSAT  Actual (January 2020)  




LANGUAGE 



LANGUAGE 



LANGUAGE 



LANGUAGE 

      AMS14    AMS22      BCN2      BER3      CBG3     

/opt/blue-python/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:182: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [4]:
site = ['AMS14','AMS22','BCN2','BER3','CBG3','LHR5','LIL2','VNO3']

final_df = pd.melt(final_df, id_vars = ['KPI','Factor','Year', 'Quarter'], value_vars = site)
final_df

final_df['sitecode'] = final_df['variable']
final_df = final_df.drop(columns = ['variable'])
final_df = final_df.fillna(0)
final_df = pd.pivot_table(final_df, values = 'value', index = ['sitecode','Factor','Year','Quarter'], columns = ['KPI'])
final_df = final_df.reset_index(level=['sitecode','Factor','Year','Quarter'])
final_df.to_csv('Q2_2020.csv')
final_df

KPI sitecode                                            Factor  Year  Quarter  \
0      AMS14  5% increment on Relocation and Incorrect Charges  2020        2   
1      AMS14                              5% increment overall  2020        2   
2      AMS14                                  Activity_Channel  2020        1   
3      AMS14                                  Activity_Channel  2020        2   
4      AMS14                            Actual (December 2019)  2019        4   
5      AMS14                                Actual Performance  2018        2   
6      AMS14                                Actual Performance  2019        2   
7      AMS14                                          Baseline  2020        2   
8      AMS14                                    Hired_Language  2020        1   
9      AMS14                                    Hired_Language  2020        2   
10     AMS14                                  Offered_Language  2020        1   
11     AMS14                                  Offered_Language  2020        2   
12     AMS14                                Scheduled_Language  2020        2   
13     AMS14                                     Shift_Pattern  2020        2   
14     AMS14                                            Target  2020        2   
15     AMS14                                            Tenure  2020        1   
16     AMS14                                            Tenure  2020        2   
17     AMS22  5% increment on Relocation and Incorrect Charges  2020        2   
18     AMS22                              5% increment overall  2020        2   
19     AMS22                                  Activity_Channel  2020        1   
20     AMS22                                  Activity_Channel  2020        2   
21     AMS22                            Actual (December 2019)  2019        4   
22     AMS22                                Actual Performance  2018        2   
23     AMS22                                Actual Performance  2019        2   
24     AMS22                                          Baseline  2020        2   
25     AMS22                                    Hired_Language  2020        1   
26     AMS22                                    Hired_Language  2020        2   
27     AMS22                                  Offered_Language  2020        1   
28     AMS22                                  Offered_Language  2020        2   
29     AMS22                                Scheduled_Language  2020        2   
..       ...                                               ...   ...      ...   
106     LIL2                            Actual (December 2019)  2019        4   
107     LIL2                                Actual Performance  2018        2   
108     LIL2                                Actual Performance  2019        2   
109     LIL2                                          Baseline  2020        2   
110     LIL2                                    Hired_Language  2020        1   
111     LIL2                                    Hired_Language  2020        2   
112     LIL2                                  Offered_Language  2020        1   
113     LIL2                                  Offered_Language  2020        2   
114     LIL2                                Scheduled_Language  2020        2   
115     LIL2                                     Shift_Pattern  2020        2   
116     LIL2                                            Target  2020        2   
117     LIL2                                            Tenure  2020        1   
118     LIL2                                            Tenure  2020        2   
119     VNO3  5% increment on Relocation and Incorrect Charges  2020        2   
120     VNO3                              5% increment overall  2020        2   
121     VNO3                                  Activity_Channel  2020        1   
122     VNO3                                  Activity_Channel  2020        2   
123     VNO3                   